## Enriching the search

In this notebook, we will define additional attributes that need to be calculated and stored during the search.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import optuna

In [3]:
# load dataset

X, y = load_breast_cancer(return_X_y=True, as_frame=True)
y = y.map({0:1, 1:0})

X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
# the target:
# percentage of benign (0) and malign tumors (1)

y.value_counts() / len(y)

target
0    0.627417
1    0.372583
Name: count, dtype: float64

In [5]:
# split dataset into a train and test set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

## Define the objective function

This is the hyperparameter response space, the function we want to optimize.

In [8]:
# the objective function takes the hyperparameter space
# as input

def objective(trial):
    
    classifier_name = trial.suggest_categorical("classifier", ["logit", "RF", 'GBM'])
    
    if classifier_name == "logit":
        
        logit_penalty = trial.suggest_categorical('logit_penalty', ['l1','l2'])
        logit_c = trial.suggest_float('logit_c', 0.001, 10)
        logit_solver = 'saga'
        
        model = LogisticRegression(
            penalty=logit_penalty,
            C=logit_c,
            solver=logit_solver,
        )
        
    elif classifier_name =="RF":
        
        rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
        rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
        rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
        rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)

        model = RandomForestClassifier(
            n_estimators=rf_n_estimators,
            criterion=rf_criterion,
            max_depth=rf_max_depth,
            min_samples_split=rf_min_samples_split,
        )
        
    else:
        
        gbm_n_estimators = trial.suggest_int("gbm_n_estimators", 100, 1000)
        gbm_criterion = trial.suggest_categorical("gbm_criterion", ['squared_error', 'friedman_mse'])
        gbm_max_depth = trial.suggest_int("gbm_max_depth", 1, 4)
        gbm_min_samples_split = trial.suggest_float("gbm_min_samples_split", 0.01, 1)

        model = GradientBoostingClassifier(
            n_estimators=gbm_n_estimators,
            criterion=gbm_criterion,
            max_depth=gbm_max_depth,
            min_samples_split=gbm_min_samples_split,
        )

    
    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    acc_std = score.std()
    
    trial.set_user_attr("accuracy", accuracy)
    trial.set_user_attr("accuracy_std", acc_std)
    
    return accuracy

## TPE

TPESampler is the default

In [9]:
study = optuna.create_study(
    direction="maximize",
    # we do not need the below line, 
    # tpe is the default search
    sampler=optuna.samplers.TPESampler(),
)


study.optimize(objective, n_trials=20)

[I 2024-09-24 12:59:30,890] A new study created in memory with name: no-name-6e0dd1a1-dc19-494a-8297-e318a740cd3b
[I 2024-09-24 12:59:35,286] Trial 0 finished with value: 0.9723551302498671 and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 745, 'gbm_criterion': 'friedman_mse', 'gbm_max_depth': 1, 'gbm_min_samples_split': 0.6462508619554381}. Best is trial 0 with value: 0.9723551302498671.
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[I 2024-09-24 13:00:40,306] Trial 19 finished with value: 0.9548112706007442 and parameters: {'classifier': 'GBM', 'gbm_n_estimators': 421, 'gbm_criterion': 'friedman_mse', 'gbm_max_depth': 3, 'gbm_min_samples_split': 0.39741149298062806}. Best is trial 0 with value: 0.9723551302498671.


In [12]:
study.best_params

{'classifier': 'GBM',
 'gbm_n_estimators': 745,
 'gbm_criterion': 'friedman_mse',
 'gbm_max_depth': 1,
 'gbm_min_samples_split': 0.6462508619554381}

In [13]:
study.best_value

0.9723551302498671

In [15]:
r = study.trials_dataframe()

r.head()

,number,value,datetime_start,datetime_complete,duration,params_classifier,params_gbm_criterion,params_gbm_max_depth,params_gbm_min_samples_split,params_gbm_n_estimators,params_logit_c,params_logit_penalty,params_rf_criterion,params_rf_max_depth,params_rf_min_samples_split,params_rf_n_estimators,user_attrs_accuracy,user_attrs_accuracy_std,state
0,0,0.972355,2024-09-24 12:59:30.891622,2024-09-24 12:59:35.286107,0 days 00:00:04.394485,GBM,friedman_mse,1.0,0.646251,745.0,NaN,NaN,NaN,NaN,NaN,NaN,0.972355,0.003599,COMPLETE
1,1,0.912015,2024-09-24 12:59:35.287106,2024-09-24 12:59:35.352510,0 days 00:00:00.065404,logit,NaN,NaN,NaN,NaN,2.749381,l2,NaN,NaN,NaN,NaN,0.912015,0.019973,COMPLETE
2,2,0.625636,2024-09-24 12:59:35.353489,2024-09-24 12:59:38.415758,0 days 00:00:03.062269,RF,NaN,NaN,NaN,NaN,NaN,NaN,gini,4.0,0.647396,697.0,0.625636,0.002229,COMPLETE
3,3,0.912015,2024-09-24 12:59:38.416737,2024-09-24 12:59:38.476705,0 days 00:00:00.059968,logit,NaN,NaN,NaN,NaN,3.848503,l2,NaN,NaN,NaN,NaN,0.912015,0.019973,COMPLETE
4,4,0.912015,2024-09-24 12:59:38.478703,2024-09-24 12:59:38.560657,0 days 00:00:00.081954,logit,NaN,NaN,NaN,NaN,0.628616,l1,NaN,NaN,NaN,NaN,0.912015,0.019973,COMPLETE
